In [1]:
import os
import pdfplumber
import numpy as np
import pandas as pd
from WindPy import w


In [2]:
w.start() # 默认命令超时时间为120秒，如需设置超时时间可以加入waitTime参数，例如waitTime=60,即设置命令超时时间为60秒

w.isconnected() # 判断WindPy是否已经登录成功


Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


True

In [3]:
#改金额单位
def mc(amount):
    value = ''
    amount = float(amount)
    if abs(amount) >= 10000 and abs(amount) < 100000000:
        val = round(amount/10000,2)
        value = str(val) + "万元"
    elif abs(amount) >= 100000000:
        val = round(amount/100000000,2)
        value = str(val) + "亿元"

    return value


# In[4]:


def file_name(file_dir):
    L=[]
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.pdf':
                L.append(os.path.join(root, file))
    return L


In [4]:
# 读取每一个pdf的股票代码
dir_name = 'D:/GF/time_saver/半年度报告生成器'
# dir_list = os.listdir(dir_name)
# print(dir_list[1])
filelist = file_name(dir_name)


tot_num_files = np.size(filelist)
codes = {}
#
for i in range(0,tot_num_files):
    if filelist[i].find('.S')!= -1 :
        a = filelist[i].find('.S') + 3
        b = filelist[i][:a]
        c = b[(a-9):]
        codes[i] = c


In [5]:
codes

{0: '300085.SZ', 1: '002813.SZ', 2: '688321.SH', 3: '002296.SZ'}

In [22]:
for num in range (0,tot_num_files):
    pdf = pdfplumber.open(filelist[num])

    #code = input("股票代码：")
    
    code = codes[num]



    # In[7]:


    # yy = x[1].find('.S') + 3
    # ww = x[1][:yy]
    # aa = ww[(yy-9):]
    # aa


    # In[8]:


    # 手动输入报表摘要所在页码
    # page_num = input("page number:")
    # page_num = int(page_num)


    # In[9]:


    # 自动查找页码
    counter = 0
    for i in range(2,10):
        p0 = pdf.pages[i]
        pp = p0.extract_text(x_tolerance=3, y_tolerance=3)
        if pp.find("主要会计数据")> -1:
            page_num = i
            break




    # In[10]:


    p0 = pdf.pages[page_num]
    table = p0.extract_table()


    # In[11]:


    table


    # In[12]:



    df = pd.DataFrame(table[1:6], columns=table[0])
    # for column in ["Effective", "Received"]:
    #     df[column] = df[column].str.replace(" ", "")
    df


    # In[ ]:





    # In[13]:


    data = df.values

    #设置变量index，保证扣非的读取
    test_var1 = data[3,0]
    test_var2 = data[2,0]
    index = []
    if test_var1!= None and test_var2.find("经常性")!= -1:
        index = int(2)
    elif test_var1==None and test_var2.find("经常性")!= -1:
        index = int(3)


    percent = {}

    #去除逗号
    for i in [0,1,index]:
        for j in [1,2,3]:
            data[i,j] = data[i,j].replace(',','')
            percent[i] = data[i,3]
            if percent[i].find('%')!=-1:
                percent[i] = percent[i].replace('%','')



    # In[14]:


    type(data[1,1])


    # In[15]:


    # 修改数据类型
    percent[0] = float(percent[0])
    percent[1] = float(percent[1])
    percent[index] = float(percent[index])

    # 判断增减
    updown = {}
    for i in [0,1,index]:
        if percent[i]>=0:
            updown[i]="增加"
        else:
            updown[i]="减少"
            percent[i]=abs(percent[i])


    # In[16]:


    Revenue = {'obj':data[0,0],'this':data[0,1],'last':data[0,2],'varpct':percent[0]} #把数据写入dict

    NetProfit_sh = {'obj':data[1,0],'this':data[1,1],'last':data[1,2],'varpct':percent[1]} #把数据写入dict

    NetProfit_sh_loss = {'obj':data[2,0],'this':data[index,1],'last':data[index,2],'varpct':percent[index]} #把数据写入dict



    # In[17]:


    line1_h1 = "20H1营业总收入" + mc(Revenue['this']) + ",上年同期为" + mc(Revenue['last']) + ",同比" + updown[0] + str(percent[0]) + '%'
    line2_h1 = "20H1归母净利润" + mc(NetProfit_sh['this']) + ",上年同期为" + mc(NetProfit_sh['last']) + ",同比" + updown[1] + str(percent[1]) + '%'
    line3_h1 = "20H1扣非归母净利润" + mc(NetProfit_sh_loss['this']) + ",上年同期为" + mc(NetProfit_sh_loss['last']) + ",同比" + updown[index] + str(percent[index]) + '%'


    # In[18]:


    ######### 以下是wind部分 ##########


    # In[19]:


    #营业总收入 _y
    # 从wind查找q1数据

    wsddata1 = w.wsd(code, "tot_oper_rev", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
    wsddata1


    # In[20]:


    #计算q2数据
    lastq1_y = wsddata1.Data[0][0] #选取2019年q1数据
    lasth1_y = wsddata1.Data[0][1] #选取2019年h1数据
    thisq1_y = wsddata1.Data[0][4] #选取2020年q1数据
    thish1_y = float(Revenue['this'])
    lastq2_y = lasth1_y - lastq1_y    #计算2019年q2数据
    thisq2_y = thish1_y - thisq1_y

    #计算同比增长/减少
    growth_year = {}
    growth_year[0] = (thisq2_y - lastq2_y)/abs(lastq2_y) * 100


    # In[21]:


    #归母净利润 _g
    # 从wind查找q1数据
    wsddata2 = w.wsd(code, "np_belongto_parcomsh", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
    wsddata2


    # In[22]:



    #计算q2数据
    lastq1_g = wsddata2.Data[0][0] #选取2019年q1数据
    lasth1_g = wsddata2.Data[0][1] #选取2019年h1数据
    thisq1_g = wsddata2.Data[0][4] #选取2020年q1数据
    thish1_g = float(NetProfit_sh['this'])
    lastq2_g = lasth1_g - lastq1_g     #计算2019年q2数据
    thisq2_g = thish1_g - thisq1_g

    #计算同比增长/减少
    growth_year[1] = (thisq2_g - lastq2_g)/abs(lastq2_g) * 100


    # In[23]:


    #扣非归母净利润 _k
    # 从wind查找q1数据
    wsddata3 = w.wsd(code, "deductedprofit", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
    wsddata3


    # In[24]:


    #计算q2数据
    lastq1_k = wsddata3.Data[0][0] #选取2019年q1数据
    lasth1_k = wsddata3.Data[0][1] #选取2019年h1数据
    thisq1_k = wsddata3.Data[0][4] #选取2020年q1数据
    thish1_k = float(NetProfit_sh_loss['this'])
    lastq2_k = lasth1_k - lastq1_k     #计算2019年q2数据
    thisq2_k = thish1_k - thisq1_k

    #计算同比增长/减少
    growth_year[2] = (thisq2_k - lastq2_k)/abs(lastq2_k) * 100


    # In[25]:


    # 判断增减
    updown_q2 = {}
    for i in [0,1,2]:
        if growth_year[i]>=0:
            updown_q2[i]="增加"
        else:
            updown_q2[i]="减少"
            growth_year[i]=abs(growth_year[i])


    # In[26]:


    line1_q2 = "20Q2营业总收入" + mc(thisq2_y) + ",上年同期为" + mc(lastq2_y) + ",同比" + updown_q2[0] + str(round(growth_year[0],2)) + '%'
    line2_q2 = "20Q2归母净利润" + mc(thisq2_g) + ",上年同期为" + mc(lastq2_g) + ",同比" + updown_q2[1] + str(round(growth_year[1],2)) + '%'
    line3_q2 = "20Q2扣非归母净利润" + mc(thisq2_k) + ",上年同期为" + mc(lastq2_k) + ",同比" + updown_q2[2] + str(round(growth_year[2],2)) + '%'


    # In[27]:

    print(codes[num])
    print(line1_h1)
    print(line2_h1)
    print(line3_h1)
    print(line1_q2)
    print(line2_q2)
    print(line3_q2)
    print('\n ')



300085.SZ
20H1营业总收入5.4亿元,上年同期为6.37亿元,同比减少15.3%
20H1归母净利润-2262.3万元,上年同期为128.16万元,同比减少1865.27%
20H1扣非归母净利润-3426.53万元,上年同期为-220.72万元,同比减少1452.43%
20Q2营业总收入2.84亿元,上年同期为2.85亿元,同比减少0.41%
20Q2归母净利润-735.19万元,上年同期为339.83万元,同比减少316.34%
20Q2扣非归母净利润-1580.99万元,上年同期为-741.02万元,同比减少113.35%

 
002813.SZ
20H1营业总收入2.93亿元,上年同期为3.04亿元,同比减少3.61%
20H1归母净利润-3365.28万元,上年同期为-4396.06万元,同比增加23.45%
20H1扣非归母净利润-4284.64万元,上年同期为-4609.99万元,同比增加7.06%
20Q2营业总收入1.78亿元,上年同期为1.4亿元,同比增加26.55%
20Q2归母净利润-2502.55万元,上年同期为-3236.95万元,同比增加22.69%
20Q2扣非归母净利润-2682.84万元,上年同期为-3377.81万元,同比增加20.57%

 


AttributeError: 'numpy.ndarray' object has no attribute 'replace'